# SQLite 기본 예제

In [1]:
import sqlite3 as sql

### DB 연결

In [5]:
# 내가 어느 db를 쓸 건지 명시해줘야 함.
conn = sql.connect('./test.db')
# connect : 원하는 db를 변수로써 할당
# db가 없을 경우, error를 출력하는 대신 새로운 db를 만든다.

# cursor : db에 있는 데이터에 순차적으로 액세스할 수 있는 일종의 포인터 역할.
cur = conn.cursor()

### 테이블 생성

In [6]:
# 테이블을 삭제하는 구문
# conn.execute('drop table product')

# 테이블 생성
conn.execute("""create table product(
    pid INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price INTEGER)""")
    
# >>> OperationalError: table product already exists

OperationalError: table product already exists

### INSERT

In [7]:
# 단일 쿼리 실행
# 변수가 들어갈 자리를 ?로 처리해놓은 후, cursor를 통해 뒤의 값으로 그 ? 자리를 대체하도록 함.
sql = 'INSERT INTO product(name, price) VALUES (?, ?)'
cur.execute(sql, ('애플 아이폰 13 프로',1300000))

In [8]:
# 동일 쿼리에 여러 데이터를 동시에 입력
cur.executemany( sql, 
    [ ('삼성 갤럭시 s21', 1200000), 
      ('삼성 갤럭시 폴드3', 2000000), 
    ]
)

In [11]:
# 여러 데이터를 동시에 입력할 때 별도로 튜플을 만들어 사용할 수도 있음.
data = (
    ('삼성 갤럭시 s21', 1200000), 
    ('삼성 갤럭시 폴드3', 2000000)
)
cur.executemany(sql,data)

### SELECT

In [3]:
# 조회
cur.execute('select * from product')

# rows : 가지고 온 데이터들이 list 형태로 반환됨.
rows = cur.fetchall()

for row in rows: 
    print(row)

(7, 'Iphone 13 pro', 1300000)
(8, 'Galaxy S22', 1100000)
(9, 'LG V50', 950000)
(11, '애플 아이폰 13 프로', 1300000)
(12, '삼성 갤럭시 s21', 1200000)
(13, '삼성 갤럭시 폴드3', 2000000)


In [4]:
# 조회 결과를 리스트가 아닌 테이블 컬럼의 dict 형태로 처리
# web 에서 view 리턴시는 rows를 그냥 리턴하면 됨
conn.row_factory = sql.Row
cur = conn.cursor()
cur.execute('select * from product')
rows = cur.fetchall()

for row in rows: 
    print(dict(row))

{'pid': 1, 'name': '애플 아이폰 13 프로 맥스', 'price': 1900000}
{'pid': 2, 'name': '삼성 갤럭시 s21', 'price': 1200000}
{'pid': 3, 'name': '삼성 갤럭시 폴드3', 'price': 2000000}


### DB연결 종료

파일을 저장할 때도, 저장하는 `save`나 `write` method가 없었기 때문에 `close` method가 사용됨.

In [4]:
# commit, rollback
# autocommit이 꺼져 있을 경우 commit을 명시해줘야 함.

conn.commit()
conn.close()

### with 구문 사용으로 auto close 구현
- 사용후 resource close 를 하지 않아도 되는 장점
- 해당 리소스를 사용하는 블럭이 길어지는 경우 코드 가독성에 문제가 있을 수 있음

In [10]:
conn = sql.connect("test.db")
with conn:
    cur = conn.cursor()
    sql = 'INSERT INTO product(name, price) VALUES (?, ?)'
    cur.execute(sql, ('애플 아이폰 13 프로 맥스',1900000))